In [1]:
import utils
from SI import SI
from SIR import SIR

SI

In [2]:
color_dict = {0:'blue',1:'orange'}#blue - healthy, orange - infected
size = 300
G, healthy, infected, pos = utils.new_graph(size=size,initial_infections=20, graph_type='nws', params={'neighbors_connected': 6, 'prob_connection': 0.1})
model = SI(G=G, pos=pos, size=size, healthy=healthy, infected=infected, color_dict=color_dict, b=0.2)
model.run_evolution()
evolution = model.get_infected_history()
t = model.get_run_time()
utils.draw_evolution(evolution, t, 'SI', 'infected_history', 'number of infected')

SIR

In [3]:
color_dict = {0:'blue',1:'orange', 2:'green'}#blue - healthy, orange - infected, green - recovered
size = 300
G, healthy, infected, pos = utils.new_graph(size=size,initial_infections=20, graph_type='nws', params={'neighbors_connected': 6, 'prob_connection': 0.1})
model = SIR(G=G, pos=pos, size=size, healthy=healthy, infected=infected, color_dict=color_dict, b=0.2, u=0.1)
model.run_evolution()
evolution_infected = model.get_infected_history()
evolution_recovered = model.get_recovered_history()
t = model.get_run_time()
utils.draw_evolution(evolution_infected, t, 'SIR', 'infected_history', 'number of infected')
utils.draw_evolution(evolution_recovered, t, 'SIR', 'recovered_history', 'number of recovered')